In [2]:
from peft_methods.vit_image import *
from easydict import EasyDict
import argparse

In [3]:
# Manually set the arguments
args = argparse.Namespace(
    batch_size=128,
    epochs=100,
    accum_iter=1,
    model='vit_base_patch16',
    weight_decay=0.0,
    lr=None,
    min_lr=0.0,
    warmup_epochs=20,
    finetune='/run/user/108646/gvfs/sftp:host=flexo/d/maboum/AdaptFormer/checkpoints/mae_pretrain_vit_b.pth',
    global_pool=False,
    data_path='/datasets01/imagenet_full_size/061417/',
    nb_classes=1000,
    output_dir='./output_dir',
    log_dir=None,
    device='cuda',
    seed=0,
    resume='',
    strategy = 'dino',
    start_epoch=0,
    eval=False,
    dist_eval=False,
    num_workers=10,
    pin_mem=True,
    world_size=1,
    local_rank=-1,
    dist_on_itp=False,
    dist_url='env://',
    dataset='cifar100',
    drop_path=0.0,
    inception=False,
    ffn_adapt=True,
    ffn_num=64,
    vpt=False,
    vpt_num=1,
    fulltune=False
)

print(args)
# Add the rest of your script logic here


Namespace(batch_size=128, epochs=100, accum_iter=1, model='vit_base_patch16', weight_decay=0.0, lr=None, min_lr=0.0, warmup_epochs=20, finetune='/run/user/108646/gvfs/sftp:host=flexo/d/maboum/AdaptFormer/checkpoints/mae_pretrain_vit_b.pth', global_pool=False, data_path='/datasets01/imagenet_full_size/061417/', nb_classes=1000, output_dir='./output_dir', log_dir=None, device='cuda', seed=0, resume='', strategy='dino', start_epoch=0, eval=False, dist_eval=False, num_workers=10, pin_mem=True, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', dataset='cifar100', drop_path=0.0, inception=False, ffn_adapt=True, ffn_num=64, vpt=False, vpt_num=1, fulltune=False)


In [ ]:
import neptune

run = neptune.init_run(
    project="continual-semantic-segmentation/peft-methods",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwN2IzOGYxMC0xYTg5LTQxMGEtYjE3Yy1iNDVkZDM1MmEzYzIifQ==",
)  # your credentials

params = {"learning_rate": 0.001, "optimizer": "Adam"}
run["parameters"] = params

for epoch in range(10):
    run["train/loss"].append(0.9 ** epoch)

run["eval/f1_score"] = 0.66

run.stop()


In [4]:
tuning_config = EasyDict(
        # AdaptFormer
        ffn_adapt=args.ffn_adapt,
        ffn_option="parallel",
        ffn_adapter_layernorm_option="none",
        ffn_adapter_init_option="lora",
        ffn_adapter_scalar="0.1",
        ffn_num=args.ffn_num,
        d_model=768,
        # VPT related
        vpt_on=args.vpt,
        vpt_num=args.vpt_num,
    )

In [ ]:
import peft_methods.vit_image as vit_image
if args.model.startswith('vit'):
        model = vit_image.__dict__[args.model](
        num_classes=args.nb_classes,
        global_pool=args.global_pool,
        drop_path_rate=args.drop_path,
        tuning_config=tuning_config,
        )
else:
    raise NotImplementedError(args.model)

In [ ]:
model

In [ ]:
import timm
timm_vision_transformer = timm.create_model('vit_base_patch16_224_dino', pretrained=True)
# Obtenir le dictionnaire d'état (state_dict) du modèle
state_dict = timm_vision_transformer.state_dict()

# Afficher les clés du dictionnaire d'état
for key in state_dict.keys():
    print(key)

In [ ]:
# Récupérer les state_dicts
state_dict_1 = model.state_dict()
state_dict_2 = timm_vision_transformer.state_dict()

# Extraire les clés et ignorer celles contenant "adaptmlp"
keys_model_1 = {key for key in state_dict_1.keys() if "adaptmlp" not in key}
keys_model_2 = {key for key in state_dict_2.keys() if "adaptmlp" not in key}

# Clés communes
common_keys = keys_model_1 & keys_model_2

# Clés différentes
diff_keys_1 = keys_model_1 - keys_model_2
diff_keys_2 = keys_model_2 - keys_model_1

print("Clés communes:")
for key in sorted(common_keys):
    print(key)

print("\nClés présentes uniquement dans le premier modèle:")
for key in sorted(diff_keys_1):
    print(key)

print("\nClés présentes uniquement dans le modèle timm:")
for key in sorted(diff_keys_2):
    print(key)

In [ ]:
state_dict = timm_vision_transformer.state_dict()
# Spécifiez la clé dont vous voulez connaître la dimension
specific_key = 'blocks.0.attn.qkv.bias'

# Vérifier si la clé existe dans le state_dict
if specific_key in state_dict:
    # Récupérer les dimensions du tensor associé à cette clé
    weight_tensor = state_dict[specific_key]
    dimensions = weight_tensor.size()  # ou weight_tensor.shape
    print(f"Dimensions des poids pour la clé '{specific_key}': {dimensions}")
else:
    print(f"La clé '{specific_key}' n'existe pas dans le state_dict.")


In [ ]:
def split_qkv(weights, biases):
    """
    Divise les poids et biais concaténés en poids et biais pour 'query', 'key', et 'value'.
    """
    total_dim = weights.size(0)
    assert total_dim % 3 == 0, "Les dimensions totales des poids ne sont pas divisibles par 3."
    qkv_dim = total_dim // 3

    # Diviser les poids
    q_weight = weights[:qkv_dim]
    k_weight = weights[qkv_dim:2*qkv_dim]
    v_weight = weights[2*qkv_dim:]

    # Diviser les biais
    q_bias = biases[:qkv_dim]
    k_bias = biases[qkv_dim:2*qkv_dim]
    v_bias = biases[2*qkv_dim:]
    return (q_weight, k_weight, v_weight), (q_bias, k_bias, v_bias)

def modify_attn_weights(state_dict):
    """
    Modifie les poids concaténés en poids individuels pour chaque bloc du state_dict.
    """
    new_state_dict = state_dict.copy()

    for key in list(state_dict.keys()):
        if key.endswith('attn.qkv.weight'):
            # Extraire les poids concaténés
            qkv_weight = state_dict[key]

            # Diviser les poids concaténés
            (q_weight, k_weight, v_weight), _ = split_qkv(
                qkv_weight,
                state_dict[key.replace('.weight', '.bias')]
            )

            # Créer les nouvelles clés pour les poids
            base_key = key.replace('.attn.qkv.weight', '.attn')
            new_state_dict[base_key + '.q_proj.weight'] = q_weight
            new_state_dict[base_key + '.k_proj.weight'] = k_weight
            new_state_dict[base_key + '.v_proj.weight'] = v_weight

            # Supprimer la clé originale
            del new_state_dict[key]

    return new_state_dict

def modify_attn_biases(state_dict):
    """
    Modifie les biais concaténés en biais individuels pour chaque bloc du state_dict.
    """
    new_state_dict = state_dict.copy()

    for key in list(state_dict.keys()):
        if key.endswith('attn.qkv.bias'):
            # Extraire les biais concaténés
            qkv_bias = state_dict[key]

            # Diviser les biais concaténés
            _, (q_bias, k_bias, v_bias) = split_qkv(
                torch.zeros_like(qkv_bias),  # Les biais sont concaténés, pas de poids associés
                qkv_bias
            )

            # Créer les nouvelles clés pour les biais
            base_key = key.replace('.attn.qkv.bias', '.attn')
            new_state_dict[base_key + '.q_proj.bias'] = q_bias
            new_state_dict[base_key + '.k_proj.bias'] = k_bias
            new_state_dict[base_key + '.v_proj.bias'] = v_bias

            # Supprimer la clé originale
            del new_state_dict[key]

    return new_state_dict

def remove_proj_keys(state_dict):
    """
    Supprime les clés de poids et biais 'proj' des blocs du state_dict.
    """
    new_state_dict = state_dict.copy()

    # Supprimer les clés de poids 'proj'
    keys_to_remove = [key for key in new_state_dict.keys() if key.endswith('attn.proj.weight')]
    for key in keys_to_remove:
        del new_state_dict[key]

    # Supprimer les clés de biais 'proj'
    keys_to_remove = [key for key in new_state_dict.keys() if key.endswith('attn.proj.bias')]
    for key in keys_to_remove:
        del new_state_dict[key]

    return new_state_dict

def modify_state_dict(state_dict):
    """
    Modifie le state_dict pour séparer les poids et biais concaténés en poids et biais individuels
    et supprimer les clés 'proj'.
    """
    state_dict = modify_attn_weights(state_dict)
    state_dict = modify_attn_biases(state_dict)
    state_dict = remove_proj_keys(state_dict)
    return state_dict


# Obtenir le dictionnaire d'état (state_dict) du modèle
state_dict = timm_vision_transformer.state_dict()

# Modifier le state_dict
modified_state_dict = modify_state_dict(state_dict)
for key in modified_state_dict.keys():
    print(key)


In [ ]:
#model.load_state_dict(modified_state_dict, strict=True)

In [ ]:
# Récupérer les state_dicts
state_dict_1 = model.state_dict()
state_dict_2 = modified_state_dict

# Extraire les clés et ignorer celles contenant "adaptmlp"
keys_model_1 = {key for key in state_dict_1.keys() if "adaptmlp" not in key}
keys_model_2 = {key for key in state_dict_2.keys() if "adaptmlp" not in key}

# Clés communes
common_keys = keys_model_1 & keys_model_2

# Clés différentes
diff_keys_1 = keys_model_1 - keys_model_2
diff_keys_2 = keys_model_2 - keys_model_1

print("Clés communes:")
for key in sorted(common_keys):
    print(key)

print("\nClés présentes uniquement dans le premier modèle:")
for key in sorted(diff_keys_1):
    print(key)

print("\nClés présentes uniquement dans le modèle timm:")
for key in sorted(diff_keys_2):
    print(key)

In [ ]:
import torch
import timm

def split_qkv(weights, biases):
    """
    Divise les poids et biais concaténés en poids et biais pour 'query', 'key', et 'value'.
    """
    total_dim = weights.size(0)
    assert total_dim % 3 == 0, "Les dimensions totales des poids ne sont pas divisibles par 3."
    qkv_dim = total_dim // 3

    # Diviser les poids
    q_weight = weights[:qkv_dim]
    k_weight = weights[qkv_dim:2*qkv_dim]
    v_weight = weights[2*qkv_dim:]

    # Diviser les biais
    q_bias = biases[:qkv_dim]
    k_bias = biases[qkv_dim:2*qkv_dim]
    v_bias = biases[2*qkv_dim:]
    return (q_weight, k_weight, v_weight), (q_bias, k_bias, v_bias)

def modify_attn_weights(state_dict):
    """
    Modifie les poids concaténés en poids individuels pour chaque bloc du state_dict.
    """
    new_state_dict = state_dict.copy()

    for key in list(state_dict.keys()):
        if key.endswith('attn.qkv.weight'):
            # Extraire les poids concaténés
            qkv_weight = state_dict[key]

            # Diviser les poids concaténés
            (q_weight, k_weight, v_weight), _ = split_qkv(
                qkv_weight,
                state_dict[key.replace('.weight', '.bias')]
            )

            # Créer les nouvelles clés pour les poids
            base_key = key.replace('.attn.qkv.weight', '.attn')
            new_state_dict[base_key + '.q_proj.weight'] = q_weight
            new_state_dict[base_key + '.k_proj.weight'] = k_weight
            new_state_dict[base_key + '.v_proj.weight'] = v_weight

            # Supprimer la clé originale
            del new_state_dict[key]

    return new_state_dict

def modify_attn_biases(state_dict):
    """
    Modifie les biais concaténés en biais individuels pour chaque bloc du state_dict.
    """
    new_state_dict = state_dict.copy()

    for key in list(state_dict.keys()):
        if key.endswith('attn.qkv.bias'):
            # Extraire les biais concaténés
            qkv_bias = state_dict[key]

            # Diviser les biais concaténés
            _, (q_bias, k_bias, v_bias) = split_qkv(
                torch.zeros_like(qkv_bias),  # Les biais sont concaténés, pas de poids associés
                qkv_bias
            )

            # Créer les nouvelles clés pour les biais
            base_key = key.replace('.attn.qkv.bias', '.attn')
            new_state_dict[base_key + '.q_proj.bias'] = q_bias
            new_state_dict[base_key + '.k_proj.bias'] = k_bias
            new_state_dict[base_key + '.v_proj.bias'] = v_bias

            # Supprimer la clé originale
            del new_state_dict[key]

    return new_state_dict


def replace_mlp_keys(state_dict):
    """
    Remplace les clés 'mlp.fc1.weight', 'mlp.fc1.bias', 'mlp.fc2.weight', 'mlp.fc2.bias'
    par 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'.
    """
    new_state_dict = state_dict.copy()

    for key in list(state_dict.keys()):
        if 'mlp.fc' in key:
            new_key = key.replace('mlp.fc', 'fc')
            new_state_dict[new_key] = new_state_dict.pop(key)

    return new_state_dict

def modify_state_dict(state_dict):
    """
    Modifie le state_dict pour séparer les poids et biais concaténés en poids et biais individuels,
    supprimer les clés 'proj', et remplacer 'mlp.fc*' par 'fc*'.
    """
    state_dict = modify_attn_weights(state_dict)
    state_dict = modify_attn_biases(state_dict)
    state_dict = replace_mlp_keys(state_dict)
    return state_dict


# Récupérer le state_dict du modèle
state_dict = timm_vision_transformer.state_dict()

# Modifier le state_dict
modified_state_dict = modify_state_dict(state_dict)



In [ ]:
for key in modified_state_dict.keys():
    print(key)

In [ ]:
#model.load_state_dict(modified_state_dict, strict=True)

In [ ]:
for key in timm_vision_transformer.state_dict().keys():
    print(key)

In [ ]:
import torch
chckpts = torch.load("/d/maboum/JSTARS/segmentation/checkpoints/sam_vit_h_4b8939.pth")

In [ ]:
for key in chckpts.keys():
    print(key)

In [1]:
from model.segmenter_adapt import *
from configs.utils import *
from datasets.utils import *
import argparse
from easydict import EasyDict
args_ = argparse.Namespace(
    batch_size=128,
    epochs=100,
    accum_iter=1,
    model='vit_base_patch16',
    weight_decay=0.0,
    lr=None,
    min_lr=0.0,
    warmup_epochs=20,
    finetune='/run/user/108646/gvfs/sftp:host=flexo/d/maboum/AdaptFormer/checkpoints/mae_pretrain_vit_b.pth',
    global_pool=False,
    data_path='/datasets01/imagenet_full_size/061417/',
    nb_classes=1000,
    output_dir='./output_dir',
    log_dir=None,
    device='cuda',
    seed=0,
    resume='',
    strategy = 'dino',
    start_epoch=0,
    eval=False,
    dist_eval=False,
    num_workers=10,
    pin_mem=True,
    world_size=1,
    local_rank=-1,
    dist_on_itp=False,
    dist_url='env://',
    dataset='cifar100',
    drop_path=0.0,
    inception=False,
    ffn_adapt=True,
    ffn_num=64,
    vpt=False,
    vpt_num=1,
    fulltune=False, 
    train_type = "finetuning",
)

tuning_config = EasyDict(
        # AdaptFormer
        ffn_adapt=args_.ffn_adapt,
        ffn_option="parallel",
        ffn_adapter_layernorm_option="none",
        ffn_adapter_init_option="lora",
        ffn_adapter_scalar="0.1",
        ffn_num=args_.ffn_num,
        d_model=768,
        # VPT related
        vpt_on=args_.vpt,
        vpt_num=args_.vpt_num,
    )
config_file = "C:/Users/marie/css-peft/configs/config.yml"
config = load_config_yaml(file_path = config_file)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = config["dataset"]
data_config = dataset["flair1"]
seed = config["seed"]
directory_path = data_config["data_path"]
metadata = data_config["metadata"]
data_sequence = data_config["task_name"]
epochs = data_config['epochs']
eval_freq = data_config['eval_freq']
im_size = data_config["im_size"]
lr = data_config['learning_rate']
win_size = data_config["window_size"]
win_stride = data_config["window_stride"]
n_channels = data_config['n_channels']
n_class = data_config["n_cls"]
class_names = data_config["classnames_binary"]
eval_freq = data_config["eval_freq"]

selected_model = '_'.join([config["model_name"], "224"])
model = config["model"]
model_config = model[selected_model]
im_size = model_config["image_size"]
patch_size = model_config["patch_size"]
d_model = model_config["d_model"]
n_heads = model_config["n_heads"]
n_layers = model_config["n_layers"]
d_encoder = model_config["d_model"]

train_type = args_.train_type
lora_params = config["lora_parameters"]
lora_rank = lora_params["rank"]
lora_alpha = lora_params["rank"]

binary = data_config["binary"]

segmentation_model = SegmenterAdapt(im_size, n_layers, d_model, d_encoder, 4 * d_model, n_heads, n_class,
                                            patch_size, selected_model, tuning_config=tuning_config,
                                            model_name=config["model_name"]).to(device)
segmentation_model.load_pretrained_weights()
for param in segmentation_model.encoder.parameters():
    param.requires_grad = False
# Unfreeze the adapt_mlp layers
for name, param in segmentation_model.encoder.named_parameters():
    if 'adaptmlp' in name:
        param.requires_grad = True

c:\Users\marie\anaconda3\envs\css\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


AttributeError: 'EasyDict' object has no attribute 'nb_task'

In [5]:
segmentation_model

SegmenterAdapt(
  (encoder): VisionTransformerAdapt(
    (patch_embed): PatchEmbedding(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=Tr